## Investigating the Efficiency of File Formats for SWOT Data on the Cloud ☁️
As terrabytes of data begin to stream into the cloud from the SWOT mission, storing and providing access to this data in the cloud has become a big priority. A new file format is needed to provide access to the data that is currently sitting on the cloud because the original format, .nc, is not efficient enough to use on cloud servers. With candidates such as Zarr and JSON via Kerchunk, this project is centered around the ease of writing, loading, and reading data to and from these various file formats.

#### Eric Pham - Jet Propulsion Laboratory 🚀

In [ ]:
'''Import statements.'''

import boto3
import json
import xarray as xr
import zarr
import s3fs
import os
import kerchunk.hdf
from kerchunk.combine import MultiZarrToZarr
import pandas as pd
import numpy as np
import fsspec
import requests
import cartopy.crs as ccrs
from matplotlib import pyplot as plt
from os import path
import hvplot.pandas
import hvplot.xarray

hvplot.extension('bokeh')
%matplotlib inline

### Part 0: Bucket Access and Dask 🪣

A dask client is used to parallelize the loading and reading of the data. The AWS key ID and secret key are also provided to allow access to data in the bucket. 

Something to make note of is that most of the processes were run on a small server notebook with a 4GB memory limit. This meant that in terms generating, loading, and plotting, certain file types were limited. In particular, using Matplotlib to display the plots overloaded memory several times so it was not a viable option for plotting data. Generating large Zarr and Kerchunk files also overloaded the server, so these processes are best done locally if large amounts of data are involved. Loading large amounts of NetCDF and Individual Kerchunk data also overloaded memory.

In [ ]:
'''Daskhub client initialization.'''
from dask.distributed import Client

client = Client(n_workers=2)
client

In [ ]:
'''AWS credentials.'''

'''
os.environ["AWS_ACCESS_KEY_ID"] = #Enter Key
os.environ["AWS_SECRET_ACCESS_KEY"] = #Enter Secret Key

s3 = s3fs.S3FileSystem(anon=False, key= #Enter Key, 
secret= #Enter Secret Key)
df = pd.read_csv(#Enter Name of Dataset).drop(columns="Unnamed: 0")
local_df = df[df["Environment"]=="Local"]
cloud_df = df[df["Environment"]=="Cloud"]
'''

### Part 1: File Generation 📁
The code blocks below provide examples on how to write the original file into a new format. Data about the speed of writing is also provided.

In [ ]:
'''Writing multiple granules to a zarr file.'''

# Edit this section and replace it with the s3path which contains the data.
num_gran = 23
s3path_nc = f's3://podaac-swot-science-sandbox/data_{num_gran}_netcdf/*'
netcdf_files = s3.glob(s3path_nc)
data_files = [s3.open(file) for file in netcdf_files]

# Opens an xarray dataset of all the granules concatenated by num_lines.
com_data = xr.open_mfdataset(data_files, concat_dim="num_lines", 
engine="h5netcdf", combine="nested")

# Writing the data to a zarr file. Edit the name of the ouput file. 
compressor = zarr.Blosc(cname='zstd', clevel=3)
encoding = {vname: {'compressor': compressor} for vname in com_data.data_vars}
#com_data.to_zarr(f"test_data_{num_gran}_zarr_combine", consolidated=True)

In [ ]:
'''Writing individual kerchunk files.'''

# Edit this section and replace it with the s3path which contains the data.
num_gran = 23
s3path_nc = f's3://podaac-swot-science-sandbox/data_{num_gran}_netcdf/*'
netcdf_files = s3.glob(s3path_nc)


# Dictionary of reference JSONs.
singles = []

# Writes reference files for each individual granule.
# Time to write is approximately 15 seconds per granule.
so = dict(anon=False, default_fill_cache=False, default_cache_type='first')

for u in ["s3://" + f for f in netcdf_files]:

    with fsspec.open(u, **so) as inf:

        h5chunks = kerchunk.hdf.SingleHdf5ToZarr(inf, u, inline_threshold=0)
        singles.append(h5chunks.translate()) 


In [ ]:
'''Writing a combined kerchunk file.'''

# This line of code creates a new dimension to concatenate all of the granules 
# along and creates a reference file which can be used to access multiple files.
mzz = MultiZarrToZarr(singles, remote_protocol="s3", 
remote_options={'anon': False}, coo_map={"z": "INDEX"}, concat_dims=["z"])
out = mzz.translate()

Preprocessing is an overlooked but equally important step when it comes to determining a good file format. If a file is relatively easy to load but takes a lot of time to create this can create a backlog when it comes to converting the files. And once the files are created, storing the files is another thing to take note of since these files will be sitting on the cloud, where storage is not free.

**NOTE:** Something important to note is that the writing times were recorded for only 23 granules as more data would overload the memory limit of the server.

In [ ]:
'''
data_1 = local_df.groupby("File Type").sum().reset_index()\
.sort_values(by="Writing Time (s)", ascending=False)
data_1["Projected Writing Time (min)"] = data_1["Writing Time (s)"] / 731 
* 10000 / 60

data_1.hvplot.bar(x="File Type", y="Projected Writing Time (min)", 
color="File Type", cmap=["chocolate", "gold", "palegreen", "dodgerblue"], 
title="Projected Time (min) to Write 10000 Granules Locally", legend=False)
'''

In [ ]:
'''
data_1["Projected Size (GB)"] = data_1["Size (MB)"] / 731 * 1000
data_1.hvplot.bar(x="File Type", y="Projected Size (GB)", 
color="File Type", cmap=["chocolate", "gold", "palegreen", "dodgerblue"], 
title="Projected Size (GB) of 10000 Granules on Cloud", legend=False)
'''

The **Individual Kerchunk** and **Combined Kerchunk** file is the clear winner in this category, sporting the fastest writing time while also taking up the least disk space. 

### Part 2: Loading... ⌛
Once the data is successfully written, it can be opened following the code blocks below. Data about the speed of loading is also provided.

In [ ]:
'''Loading a netcdf.'''

# Edit this section and replace it with the s3path which contains the data.
s3path_nc = f's3://podaac-swot-science-sandbox/data_{num_gran}_netcdf/*'
netcdf_files = s3.glob(s3path_nc)
netcdf_fileset = [s3.open(file) for file in netcdf_files]

# Loads the data using xarray.
netcdf_file = xr.open_mfdataset(netcdf_fileset, engine='h5netcdf', 
combine='nested', chunks={}, concat_dim="num_lines", decode_times=False)

In [ ]:
'''Loading a zarr.'''

# Edit this section and replace it with the s3path which contains the data.
gran = 23
s3path_zarrc = f's3://podaac-swot-science-sandbox/data_{gran}_zarr_combine/'
store = s3fs.S3Map(root=s3path_zarrc, s3=s3, check=False)

# Loads the data and chunks it. Chunks are based off of simulated SWOT data.
zarr_combine_file = xr.open_zarr(store=store, 
consolidated=True).chunk({"num_lines": 9864, "num_pixels": 71})

In [ ]:
'''Loading individual kerchunk files.'''

# List of opened files.
cat_files = []

# Enter AWS credentials here to access files.
remote = {"anon": False, "key": , 
    "secret": }

# Individually opens reference files.
for s in singles:

    # File access options.
    storage_i = {"fo": s, "remote_protocol": "s3", "remote_options": remote} 
    backend_i = {"storage_options": storage_i, "consolidated": False}

    # Reading the JSONs stored in singles. Alternatively, the kerchunk files can 
    # be written to a path and read in that way.
    cat_files.append(xr.open_dataset("reference://", engine="zarr", chunks={}, 
    backend_kwargs=backend_i))



# Concatenates all of the individually opened kerchunk files into a dataset.
kerchunk_files = xr.concat(cat_files, "num_lines")

In [ ]:
'''Loading combined kerchunk file.'''

# Uses AWS credentials to access file.
remote = {"anon": False, "key": , 
    "secret": }
storage_c = {"fo": out, "remote_protocol": "s3", "remote_options": remote}
backend_c = {"storage_options": storage_c, "consolidated": False}

# Opens the dataset using the combined kerchunk reference file.
combined_kerchunk_files = xr.open_dataset("reference://", engine="zarr", 
chunks={}, backend_kwargs=backend_c)
combined_kerchunk_files = combined_kerchunk_files.assign_coords(z=np.arange(23))

However, once a file is written, it does not need to be adjusted any further. The same cannot be said for loading, which makes it one of the most important metrics of performance in terms of evaluating a files performance. If the overhead time to write a file is ~5 seconds but takes 30 minutes to load everytime the dataset is needed, then it is not the most optimal means of storing data. 

All of the data collected was gathered on a cloud environment, using a notebook with a 4GB memory maximum. NetCDF and Individual Kerchunk both exceeded the memory maximum for 589 granules so no data is available for those files.

In [ ]:
'''
data_2 = cloud_df[cloud_df["Num Granules"]!=589][["File Type", 
"Loading 5 Times (s)", "Loading 10 Times (s)"]].groupby("File Type").sum()
data_2["Average Loading Time (s)"] = (data_2["Loading 5 Times (s)"] 
+ data_2["Loading 10 Times (s)"]) / 15
data_2 = data_2 / 142 * 1000
data_2["Order"] = [3, 2, 0, 1]
data_2 = data_2.sort_values(by="Order").reset_index()

data_2.hvplot.bar(x="File Type", y="Average Loading Time (s)", 
color="File Type", cmap=["chocolate", "gold", "palegreen", "dodgerblue"], 
title="Projected Loading Time (s) for 1000 Granules on Cloud", legend=False)
'''

The **Zarr** and **Combined Kerchunk** are the file formats that allow data to be access the most quickly.

### Part 3: Reading and Plotting 📍

Once the data is lazily loaded, ensuring that using the data is easy and efficient is the important final step. Blocks of code below show how to use hvplot to display data.

In [ ]:
'''Select a file format from above to plot.'''

# Sets the dataset to an already loaded dataset.
dataset = zarr_combine_file
dataset

In [ ]:
'''Plotting figures using hvplot.'''

# Plots simulated_true_ssh_karin.
dataset.simulated_true_ssh_karin.hvplot.points('longitude', 'latitude', 
aggregator="mean", crs=ccrs.PlateCarree(), projection=ccrs.PlateCarree(), 
project=True, geo=True, rasterize=True, coastline=True, frame_width=800, 
dynamic=False)

Ensuring that data is not only quick to load but also read from is important for the end-product. As of right now, the memory limit prevents plotting more ~150 granules so only a small amount of granules were test.

In [ ]:
'''
data_3 = cloud_df[cloud_df["Num Granules"]==23]

data_3.hvplot.bar(x="File Type", y="Loading + Hvplot (s)", 
color="File Type", cmap=["chocolate", "gold", "palegreen", "dodgerblue"], 
title="Time to Load + Hvplot Plot (s) 23 Granules on Cloud", legend=False)
'''

The **Zarr**, **Individual Kerchunk**, and **Combined Kerchunk** files perform equally. Compared to the NetCDF format, the difference is clear.

### Part 4: Conclusion ✅

After examining the data from all four file formats one thing remains very clear; the legacy NetCDF is not made to perform in cloud environments. However, selecting the optimal file format becomes tricky when all things are considered. The two front-runners are the **Zarr** and **Combined Kerchunk** formats. 

While the Zarr format is extremely space inefficient, it performs well in all other categories and also meshes extremely well with any existing code. It does not alter the structure of the data in any way and works well with the existing PO.DAAC tutorial code.

On the otherhand, the Combined Kerchunk format is extremely space efficient and loads and reads extremely fast. While it seems that there are no downsides to this, the Combined Kerchunk requires that a new dimension is added to the data to concatenate all the granules together. This slightly alters the structure of the data, making it difficult to plot using the existing code provided on the PO.DAAC cookbook.

These things must be considered in conjunction when making a decision as to which file format is most efficient. Zarr most closely mirrors the legacy format and is easy to manufacture whilst the Kerchunk library may pose difficults for first time users as well as slightly changing the structure of the data.


List of Resources:
- https://podaac.github.io/tutorials/external/Direct_Access_SWOT_sim_Oceanography.html
- https://fsspec.github.io/kerchunk/test_example.html
- https://github.com/lsterzinger/cloud-optimized-satellite-data-tests
- https://ntrs.nasa.gov/api/citations/20200001178/downloads/20200001178.pdf